In [2]:
%matplotlib qt5
# %matplotlib notebook

import time
import os
from datetime import datetime

import numpy as np

import matplotlib.pyplot as mplt
import matplotlib.gridspec as mgs

from siriuspy.search import IDSearch
from siriuspy.devices import SOFB, HLFOFB, Tune, CurrInfoSI, RFGen, VPU,\
     EVG, InjCtrl
from apsuite.commisslib.meas_ac_orm import MeasACORM

from mathphys.functions import save, load

mplt.rcParams.update({
    'font.size': 12,'lines.linewidth': 2,
    'axes.grid': True, 'grid.alpha': 0.5,
    'grid.linestyle': '--', 'text.usetex': True*0})

# Define functions

In [59]:
def initialize_vpu(beamline):
    # Search ID
    devnamevpu = IDSearch.conv_beamline_2_idname(beamline=beamline)
    vpu = VPU(devname=devnamevpu)

    # Disable beamline control
    vpu.cmd_beamline_ctrl_disable()
    print('beamline control: ', vpu.is_beamline_ctrl_enabled)

    # Set gap speed
    vpu.set_gap_speed(1.0)
    time.sleep(0.5)
    print('gap speed: {:.3f} mm/s'.format(vpu.gap_speed))

    return vpu

def move_vpu_gap(vpu:VPU, gap, timeout, verbose=False):
    vpu.set_gap(gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(vpu.gap)) if verbose else 0
    if vpu.cmd_move_gap_start(timeout):
        time.sleep(0.5)
        print('Undulator is moving...') if verbose else 0
        while vpu.is_moving:
            time.sleep(0.1)
            print('Current gap {:.3f} mm.'.format(vpu.gap_mon), end='\r') if verbose else 0
        print('Gap {:.3f} mm reached.'.format(vpu.gap)) if verbose else 0
        return True
    else:
        print('Error while cmd_move_start.')
        return False

def move_vpu_gap_robust(vpu:VPU, gap, timeout, maxiter=3, verbose=False):
    sucess = move_vpu_gap(vpu, gap=gap, timeout=timeout, verbose=verbose)
    i=0
    while not sucess and i<maxiter:
        i += 1
        print('Trying {:0f}/{:.0f}'.format(i, maxiter)) if verbose else 0
        vpu.cmd_reset(timeout=timeout)
        time.sleep(0.5)
        sucess = move_vpu_gap(vpu, gap=gap, timeout=timeout, verbose=verbose)
    if sucess:
        print('Movimentation done!\n')
        return True
    else:
        print('Error while moving.\n')
        return False

In [141]:
def meas_injeff(devices, nr_pulses=5):
    evg = devices['evg']
    currinfo = devices['currinfo']
    injeffs = []
    cnt = nr_pulses
    inj0 = currinfo.injeff
    while len(injeffs) < nr_pulses:
        evg.cmd_turn_on_injection()
        evg.wait_injection_finish()
        time.sleep(3)
        j = evg.bucketlist[0] 
        j += 29
        bucket = j - (j//864) * 864
        bucket = 864 if not bucket else bucket
        evg.bucketlist = [bucket]
        injn = currinfo.injeff
        if inj0 != injn:
            inj0 = injn
            injeffs.append(injn)
            cnt -= 1
        if cnt == 0:
            break
    return injeffs

def get_injeff_data(devices, nr_acq=5, nr_pulses=5, acq_sleep=1):

    curr_data = np.zeros(nr_acq)
    lifetime_data = np.zeros(nr_acq)
    
    print('measuring injeff...', end='\r')
    injeffs = meas_injeff(devices, nr_pulses)

    print('measuring tunes...', end='\r')
    for i in range(nr_acq):
        time.sleep(acq_sleep)
        curr_data[i] = devices["currinfo"].current
        lifetime_data[i] = devices["currinfo"].lifetime
        
    return curr_data, injeffs, lifetime_data

def save_injeff_data(devices, injeff_data):
    vpu = devices['vpu']
    data = dict()
    data['timestamps'] =  time.time()
    data['gaps'] = (vpu.gap_mon)
    data["current"] = injeff_data[0]
    data['inj_effs'] = injeff_data[1]
    data['lifetimes'] = injeff_data[2]

    data['beamlines'] = (IDSearch.conv_idname_2_beamline(vpu.devname))
    
    vpu_gap = '{:.2f}'.format(vpu.gap_mon).replace(".", "p")
    save(data, f'injeff_VPU_gap_{vpu_gap}.pickle')    


In [62]:
def create_devices():
    devices = {}
    devices["currinfo"] = CurrInfoSI()
    devices["tune"] = Tune()
    devices["vpu"] = initialize_vpu("CARNAUBA")
    devices["sofb"] = SOFB(SOFB.DEVICES.SI)
    devices["fofb"] = HLFOFB(HLFOFB.DEVICES.SI)
    devices['rfgen'] = RFGen()
    devices["evg"] = EVG()
    devices["injctrl"] = InjCtrl()

    evg = devices['evg']
    evg.set_nrpulses(1)
    evg.bucketlist = [1]
    
    return devices

# Create objects

In [63]:
devices = create_devices()

beamline control:  False
center mode:  0
pitch mode:  0
gap speed: 0.100 mm/s
gap: 24.000 mm
beamline control:  False
center mode:  0
pitch mode:  0
gap speed: 0.100 mm/s
gap: 24.000 mm


# Initialize objects

In [64]:
# Set injection mode to decay
timeout = 3
vpu = devices["vpu"]

In [66]:
# test measurement
injeffs = meas_injeff(devices, nr_pulses=8)

# Collect injeff data

## Get injeff data

In [71]:
injeff_data = get_injeff_data(devices, nr_acq=5, nr_pulses=8, acq_sleep=1)

## Save

In [78]:
save_injeff_data(devices, injeff_data)